In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
save_path = '/content/gdrive/MyDrive/pytest_img/_generated_images'
# _generated_images 폴더 밑에 MNIST 라는 폴더를 만든다
if not os.path.exists(os.path.join(save_path, 'MNIST/')):
  os.makedirs(os.path.join(save_path, "MNIST/"))

In [4]:
# 생성자는 이미지를 만드는 것이 목적
# 랜덤한 픽셀 값으로 채워진 가짜 이미지 생성으로부터 시작
generator = Sequential()
# 은닉 1층을 128*7*7 개로 만듦
# 128은 노드의 크기를 위한 차원의 수로서 변경 가능. 데이터가 다양한 정보를 갖게 하기 위하여 충분히 크게 함
# 7*7은 이미지의 최종 크기를 고려한 수. 뒤에서 이미지를 2배로 키우는 UpSampling을2번 거치므로 7을 사용하면 7*2*2=28이 되어 original 이미지의 28과 같게 된다
# 7*7의 수를 변경해도 괜찮지만, Reshape의 결과와 연결되므로 가능한 왜곡이 없게 하려면 은닉층의 수를 이에 맞추는 것이 좋다
# input_shape(100,)은 입력층의 shape. 가상의 이미지를 이 크기로 만든다. 일반적으로 사용하는 값이나, 변경 가능
generator.add(Dense(128*7*7, input_shape=(100,), activation=LeakyReLU(0.2)))
# 배치 정규화 레이어. 입력 데이터의 평균이 0, 분산이 1이 되도록 값을 일정하게 재배치
# 배치 정규화는 많은 경우 중요한 단계 중 하나로 인식되는데, DCGAN 개발자들은 이 레이어를 통해 안정적인 학습이 가능하다며 특히 중요하게 생각함
generator.add(BatchNormalization())

# Conv2D()에 들어갈 수 있도록 Reshape 레이어를 사용하여 데이터의 shape를 변경(1D → 3D)
generator.add(Reshape((7, 7, 128)))
# UpSampling2D()는 height와 width의 크기를 각각 2배로 늘림
# (batch_size, height, width, channel) → (batch_size, height*2, width*2, channel)
# (batch_size, 7, 7, 128) → (batch_size, 7*2, 7*2, 128)
generator.add(UpSampling2D())
# Convolution 레이어. padding=‘same’, 필터 크기 (5, 5)인 64개의 필터를 통과하면서. shape는 (14, 14, 64)
generator.add(Conv2D(64, kernel_size=5, padding='same'))
# 배치 정규화 레이어
generator.add(BatchNormalization())

# LeakyReLU()는 음수에서 값이 무조건 0이 되는 것을 방지하는 것으로, 0 이하에서도 작은 값을 갖게 함
# 여기에서는 0보다 작을 경우에는 0.2를 곱하게 한다
# GAN에서는 기존에 사용하던 ReLU() 함수를 사용하면 학습이 불안정해지는 경우가 많다
generator.add(Activation(LeakyReLU(0.2)))
# (batch_size, 14, 14, 64) → (batch_size, 14*2, 14*2, 64)
generator.add(UpSampling2D())
# Convolution 레이어. 필터 크기 (5, 5). activation 함수로 tanh를 사용하여 출력값을 -1 ~ 1 사이가 되게 함
# 훈련 시 사용되는 입력 데이터는 -1 ~ 1 사이의 값으로 정규화하여 입력되므로, 훈련 중엔 계속 이 값이 유지되게 한다
# 1필터를 갖는 Conv2D 이므로 (28, 28, 1) 이 된다
generator.add(Conv2D(1, kernel_size=5, padding='same', activation='tanh'))

In [5]:
generator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6272)              633472    
                                                                 
 batch_normalization (Batch  (None, 6272)              25088     
 Normalization)                                                  
                                                                 
 reshape (Reshape)           (None, 7, 7, 128)         0         
                                                                 
 up_sampling2d (UpSampling2  (None, 14, 14, 128)       0         
 D)                                                              
                                                                 
 conv2d (Conv2D)             (None, 14, 14, 64)        204864    
                                                                 
 batch_normalization_1 (Bat  (None, 14, 14, 64)        2

In [7]:
# 판별자는 이미지의 진위 여부를 가리는 것이 목적
discriminator = Sequential()
# 크기 (5, 5)의 필터 64개 사용. strides=2(마스크를 2칸씩 이동)
# 기본값은 1칸인데, 2칸을 움직이게 하면 height, width 크기가 더 줄어들면서 새로운 특징을 뽑아주는 효과. 즉, 새로운 필터를 적용한 효과가 생김
# 생성자에서 최종 출력된 크기가 28*28 이기 때문에 input_shape를 28*28로 함
# 색상이 아닌, 형상만 파악하면 진위 여부를 구별할 수 있으므로 채널은 1채널만 사용
discriminator.add(Conv2D(64, kernel_size=5, strides=2, input_shape=(28, 28, 1), padding='same'))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Conv2D(128, kernel_size=5, strides=2, padding='same'))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))

# 판별, 즉 분류를 하기 위하여 1차원 데이터로 변환하는 Flatten() 수행
discriminator.add(Flatten())
# 진짜(1), 가짜(0) 중 하나가 되어야 하므로 출력노드를 1로 만들고, Sigmoid 함수를 사용
discriminator.add(Dense(1, activation='sigmoid'))
# discriminator를 컴파일하여 독립적인 모델로 사용할 수 있게 한다.
discriminator.compile(loss='binary_crossentropy', optimizer='adam')
# 판별자의 학습 가능 여부를 False로 설정
# 이렇게 하면 생성자와 판별자를 함께 학습하는 GAN 모델을 컴파일 할 때는 판별자의 가중치는 업데이트되지 않고, 생성자만 학습됨
# 판별자는 가중치 업데이트는 하지 않지만, 여전히 compile시 설정된 손실함수를 사용하여 손실값은 계산함
# 계산된 손실값은 GAN 모델 전체를 통해 생성자에게 전달됨. 생성자는 전달된 판별자의 가중치를 통해 자신의 가중치를 업데이트 함
# 만약 판별자가 같이 업데이트 되면 생성자는 판별자를 속일 수 없게 되므로, 생성자가 판별자를 속일 수 있도록 판별자의 가중치 업데이트를 막는 것
discriminator.trainable = False

In [8]:
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 14, 14, 64)        1664      
                                                                 
 activation_1 (Activation)   (None, 14, 14, 64)        0         
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 7, 7, 128)         204928    
                                                                 
 activation_2 (Activation)   (None, 7, 7, 128)         0         
                                                                 
 dropout_1 (Dropout)         (None, 7, 7, 128)         0         
                                                                 
 flatten (Flatten)           (None, 6272)             

In [9]:
# 생성자는 100 차원의 크기를 갖는 랜덤 노이즈 벡터를 입력으로 받음
ginput = Input(shape=(100,))
# 생성자에서 만들어진 데이터를 판별자에 입력시킴
# 판별자를 거친 결과(손실함수 계산값)가 dis_output에 저장됨
dis_output = discriminator(generator(ginput))
# inputs에는 랜덤으로 생성된 데이터가 생성자의 결과를 거친 ginput을, outputs에는 판별자를 거친 dis_output을 입력
# dis_output은 discriminator(generator(ginput)) 과정을 거친 것이므로,
# Functional API의 Model 클래스는 생성자와 판별자를 연결하여 하나의 새로운 모델을 만든다
# 즉, 생성자로 생성된 이미지를 판별자로 판별하는 모델
gan = Model(inputs=ginput, outputs=dis_output)
# 참과 거짓을 구분해야 하므로 손실함수는 binary_crossentropy
# discriminator.trainable = False 이후 전체 모델인 gan 모델을 컴파일 하는 것이므로, 이것을 사용할 때는 학습 기능이 꺼진 discriminator가 사용된다
gan.compile(loss='binary_crossentropy', optimizer='adam')

In [10]:
gan.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential (Sequential)     (None, 28, 28, 1)         865281    
                                                                 
 sequential_1 (Sequential)   (None, 1)                 212865    
                                                                 
Total params: 1078146 (4.11 MB)
Trainable params: 852609 (3.25 MB)
Non-trainable params: 225537 (881.00 KB)
_________________________________________________________________


In [11]:
d_loss = [] # 판별자의 오차 변수
g_loss = [] # 생성자의 오차 변수

In [13]:
# GAN 모델을 훈련시키는 과정
def gan_train(epoch, batch_size, saving_interval):
  # MNIST 훈련 데이터 불러오기. 훈련데이터의 독립변수만 사용한다
  (X_train, _), (_, _) = mnist.load_data()
  # (X_train의 행의 크기, 28, 28, channel==1)
  X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')

  # 정규화 과정. 중앙값 127.5를 빼준 뒤 127.5로 나눠서 -1~1 사이의 값으로 바꿈
  # GAN에서는 -1~1의 값을 갖는 tanh를 사용하므로 이 범위를 갖도록 정규화하는 것
  X_train = (X_train-127.5) / 127.5
  true = np.ones((batch_size, 1)) # 배치 사이즈만큼의 행을 갖는 True(1) 레이블 생성
  fake = np.zeros((batch_size, 1)) # 배치 사이즈만큼의 행을 갖는 False(zero) 레이블 생성
  for i in range(epoch): # 에포크 단위 학습
    # 실제 이미지를 판별자에 입력하여 학습
    # np.random.randint(a, b, c)는 a부터 b까지의 숫자 중 하나를 랜덤으로 뽑는 과정을 c 번 반복
    # X_train.shape는 (60000, 28, 28)이므로 0부터 60,000개의 훈련데이터 중 하나 가져오기를 batch_size 만큼 반복
    idx = np.random.randint(0, X_train.shape[0], batch_size)

    # 위에서 선택된 숫자 idx를 이용하여 이미지들을 불러 옴
    imgs = X_train[idx]

    # train_on_batch(x, y) 함수는 입력값(x)과 레이블(y)을 받아서 바로 학습을 실시해 모델을 업데이트
    # 즉 batch_size 크기의 imgs 데이터를 받아 바로 학습하고 모델 업데이트. discriminator.compile() 단계의 판별자를 사용한다(훈련가능)
    # return 값은 training loss
    # imgs = X_train[idx] 에서 뽑힌 이미지를 x(진짜)에 넣고,
    # true = np.ones((batch_size, 1)) 에서 만든 배열을 y에 넣어 그것들이 진짜(1)임을 알려줌
    d_loss_real = discriminator.train_on_batch(imgs, true)

    # 가상 이미지를 판별자에 입력하여 학습. 0부터 1까지의 실수 중 2D (batch_size, 100)개를 랜덤으로 뽑음
    # (batch_size, 100)은 (batch_size 행, 100 열)의 구조로 랜덤값을 뽑음
    # 100 열인 이유는 생성자에서 100 차원 입력을 요구하였기 때문
    noise = np.random.normal(0, 1, (batch_size, 100))

    # (batch_size, 100) 구조의 랜덤값(가상 이미지)이 Generator에 들어가고, 결괏값이 gen_imgs로 저장됨
    # gen_imgs는 랜덤하게 만든 noise 가상 이미지를 generator에 넣어 생성된 이미지(tanh의 결과이므로 -1 ~ 1)
    # predict() 함수를 사용하였지만, generator의 마지막 출력층이 Conv2D 로서 진짜/가짜 이미지일 확률이 아니라,
    # generator 모델을 통해 생성된 이미지임
    # noise의 shape (batch, 100) → gen_imgs의 shape (batch, 28, 28, 1) 로 변환됨
    gen_imgs = generator.predict(noise)

    # train_on_batch()로 학습 및 모델 업데이트. discriminator.compile()의 판별자를 사용한다(훈련가능)
    # 판별자에게 gen_imgs가 모두 가짜(0) 임을 알려주어 훈련시킴
    # 생성자는 지속적으로 가중치가 업데이트되어 이전 epoch에서 만들어내는 이미지가 점점 진짜와 같아지므로 판별자의 성능도 올라감
    # 판별자는 출력층이 Dense() 층 이진분류로서 판정을 하는 데 사용됨
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)

    # 판별자와 생성자의 오차 계산. d_loss는 판별자의 성능 (손실값)
    # 실제 이미지를 넣은 결과에 대한 오차값이 d_loss_real, 가상 이미지를 넣은 결과에 대한 오차값이 d_loss_fake
    # 둘은 모두 판별자에 대한 성능이므로 이 둘의 평균 오차값을 구하여 판별자의 오차(성능)를 구함
    # d_loss_real과 d_loss_fake를 더한 뒤 1/2
    d_loss.append(0.5 * np.add(d_loss_real, d_loss_fake))

    # 판별자 가중치를 생성자로 전달하는 과정
    # train_on_batch()로 학습 및 모델 업데이트. 앞에서 생성한 GAN 모델 사용
    # g_loss는 생성자의 성능 (손실값)
    # gan.train_on_batch(noise, true)는 판별자에게 가상 이미지 noise에 대하여 true(1)라고 거짓으로 알려주고 판별자로 넘김
    # gan 모델 내부에서는 noise → generator에서 28*28*1 의 이미지로 변환 → 판별자로 판별 과정을 거침
    # 판별자는 생성자가 만든 가짜를 진짜(true(1))로 인식하려고 하지만, 초기의 가짜는 진짜와 거리가 멀어 모델의 손실값이 커지게 됨
    # 이 손실을 최소화화기 위해 생성자는 가짜 이미지를 더욱 진짜 이미지처럼 만들게 하는 쪽으로 Conv2D의 필터 가중치가 업데이트 되고,
    # 판별자는 가짜 이미지를 진짜로 판정하는 쪽으로 가중치 업데이트를 해야 하나, trainable = False 하였으므로 판별자는 업데이트 되지 않음
    # 즉, 전체 Model은 판별 결과(확률값)가 여기서 보낸 true(1)과 얼마나 차이나는지를 파악하여 그것이 보정되도록 Model을 업데이트 하는 것
    g_loss.append(gan.train_on_batch(noise, true))

    # 지금까지 업데이트 된 모델로 이미지 생성
    # saving_interval로 지정된 때마다 아래의 과정을 진행하여 만든 이미지를 저장
    if i % saving_interval == 0:
      noise = np.random.normal(0, 1, (25, 100)) # (25 행, 100 열)의 가상 이미지 생성
      gen_imgs = generator.predict(noise) # 생성자로 이미지 변환 (32, 100) → (32, 28, 28, 1)
      # -1 ~ 1 사이의 값을 갖는 이미지를 0 ~ 1 사이의 값으로 변환
      # 만들어진 gen_imgs는 (25, 28, 28, 1)
      gen_imgs = 0.5 * gen_imgs + 0.5 # 생성자 함수 마지막층이 -1~1 범위를 갖는 tanh이므로, 0~1로 변환
      fig, axs = plt.subplots(5, 5) # 생성된 25행의 가상 이미지를 5*5 로 표현할 준비
      count = 0
      for j in range(5):
        for k in range(5):
          # 25개씩 이미지 출력. (j, k) : (0, 0) → (0, 1) → (0, 2) → ... (4, 4)
          # count는 0 ~ 24까지 올라가 총 25장이 출력되며, :, : 에 의해 각 이미지가 가지고 있는 28 픽셀은 모두 출력됨
          axs[j, k].imshow(gen_imgs[count, :, :, 0], cmap='gray')
          axs[j, k].axis('off')
          count += 1
          fig.savefig(os.path.join(save_path, "MNIST/")+"gan_mnist_%d.png" % i)

In [ ]:
gan_train(40001, 32, 200)

1/1 [==============================] - 0s 139ms/step


<ipython-input-13-ba70b8a52ed4>:72: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, axs = plt.subplots(5, 5) # 생성된 25행의 가상 이미지를 5*5 로 표현할 준비


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
1/1 [==============================] - 0s 178ms/step


In [ ]:
import matplotlib.pyplot as plt
epochs = range(0, 40001)
plt.plot(epochs, g_loss, 'r', label='generator loss', linewidth=0.3)
plt.plot(epochs, d_loss, 'b', label='discriminator loss', linewidth=0.3)
plt.title("Generator and Discriminator loss")
plt.legend()
plt.show()